# COMPILER USAGE NOTEBOOK
### Input file exemple
Please enter in between the quotes of variable string the wanted program
All this code will do is take that wanted string and save it in a given file named "newfile.txt"
Change the filename as you wish

In [3]:
string = """
#TIMESCALE
horizon: 3
step: 1

#NODE ABC
#PARAMETERS
c = 10 +10.01
v = (5)**c
#VARIABLES 
internal : a
output : b
#CONSTRAINTS
a[0]=1
a[t+1] = 3*a[0]
b[t*c+1] = 3*a
b[t]-(3*b[t+1] + (-a[t]))=-2*b[t]+1 
b-(3*b[t+1] + (-a))=-2*b+1 

#OBJECTIVE
min : a

#LINKS
A = B,C
A.pay = B.day"""
f = open("newfile.txt", "w")
f.write(string)
f.close()

### Lexer
The lexer takes as input a file seen as a stream of characters and converts into a stream of tokens
The tokens can be seen below with each token being the token name, Value, Line at which it was declared and the total number of characters read before the first character of this token


In [7]:
from lexer import tokenize_file

tokenize_file("newfile.txt")

LexToken(TIME,'#TIMESCALE',40,1)
LexToken(HORIZON,'horizon',41,12)
LexToken(COLON,':',41,19)
LexToken(INT,3,41,21)
LexToken(STEP,'step',42,23)
LexToken(COLON,':',42,27)
LexToken(INT,1,42,29)
LexToken(NODE,'#NODE',44,32)
LexToken(NAME,'ABC',44,38)
LexToken(PARAM,'#PARAMETERS',45,42)
LexToken(ID,'c',46,54)
LexToken(EQUAL,'=',46,56)
LexToken(INT,10,46,58)
LexToken(PLUS,'+',46,61)
LexToken(FLOAT,10.01,46,62)
LexToken(ID,'v',47,68)
LexToken(EQUAL,'=',47,70)
LexToken(LPAR,'(',47,72)
LexToken(INT,5,47,73)
LexToken(RPAR,')',47,74)
LexToken(POW,'**',47,75)
LexToken(ID,'c',47,77)
LexToken(VAR,'#VARIABLES',48,79)
LexToken(INTERNAL,'internal',49,91)
LexToken(COLON,':',49,100)
LexToken(ID,'a',49,102)
LexToken(OUTPUT,'output',50,104)
LexToken(COLON,':',50,111)
LexToken(ID,'b',50,113)
LexToken(CONS,'#CONSTRAINTS',51,115)
LexToken(ID,'a',52,128)
LexToken(LBRAC,'[',52,129)
LexToken(INT,0,52,130)
LexToken(RBRAC,']',52,131)
LexToken(EQUAL,'=',52,132)
LexToken(INT,1,52,133)
LexToken(ID,'a',53,135)
LexToke

### Parser
#### Classes
Takes as input the token stream and converts it to various a program class containing all the necessary information about the input file. 

The Program file is made up of : 
- A list of node objects
- A list of Links
- Information about time contained in timeclass

The Node object is made up of : 
- A name
- A list of Parameter objects
- A list of Variables 
- A list of Constraints 
- A list of Objective functions

A Parameter object contains:
- A name (parameter name)
- An expression (the rhs of the equality)

A Variable contains : 
- A name (Variable name)
- A parameter Type (Internal - Input - Output)

A constraint contains :
- A rhs expression 
- A sign (between "=","<=","<",">=",">")
- A lhs expression

An objective is made up of:
- A objective function between min or max
- An expression

An expression object contains : 
- A type between : 
    - 'literal' if it is a identifier, a float or an integer
    - '*' '/' '+' '-' if it is the according operation between two expression
    - 'u-' if it is a unary minus
- A list of child expressions
- A name containing the according identifier, float or integer if its type is literal

An expression object is build up as a tree of expression as follows, it starts from the bottom and goes upward:
- An integer, float or identifier is turned into a 'literal' type expression with the expression name containing the corresponding value
- If a unary minus is applied over an expression: 
    - A new expression 'u-' is created with only one child
- If a sum,division,soustraction,multiplication is applied between two expressions:
    - A new expression with the correspond type defined as the sign of the operation and two child expression
- If an expression is contained between parenthesis, simply the parenthesis are ignored

The priority of operations is given by a predefined precedence rule which corresponds to the usual mathematical precedence

#### Code
In the following code, the 'newfile.txt' file goes through the lexer and the corresponding stream is given to the parser which following a given grammar creates a full Program object. The full program object is printed. A more comprehensive way of seeing the different components of an object from the program class can be obtained by using the function 'to_string()'

In [8]:
from Myparser import parse_file


result = parse_file("newfile.txt")
print(result)
print("\n")
print(result.to_string())

[[ [ABCD , [ [c , [+ , [ 10 10.01]]] [v , [** , [ 5 c]]]] , [ [a , internal] [b , output]] , [ [= , a[0] , 1] [= , a[[+ , [ t 1]]] , [* , [ 3 a[0]]]] [= , b[[+ , [ [* , [ t c]] 1]]] , [* , [ 3 a]]] [= , [- , [ b[t] [+ , [ [* , [ 3 b[[+ , [ t 1]]]]] [u- , [ a[t]]]]]]] , [+ , [ [* , [ [u- , [ 2]] b[t]]] 1]]] [= , [- , [ b [+ , [ [* , [ 3 b[[+ , [ t 1]]]]] [u- , [ a]]]]]] , [+ , [ [* , [ [u- , [ 2]] b]] 1]]]] , []] [ABC , [ [c , [+ , [ 10 10.01]]] [v , [** , [ 5 c]]]] , [ [a , internal] [b , output]] , [ [= , a[0] , 1] [= , a[[+ , [ t 1]]] , [* , [ 3 a[0]]]] [= , b[[+ , [ [* , [ t c]] 1]]] , [* , [ 3 a]]] [= , [- , [ b[t] [+ , [ [* , [ 3 b[[+ , [ t 1]]]]] [u- , [ a[t]]]]]]] , [+ , [ [* , [ [u- , [ 2]] b[t]]] 1]]] [= , [- , [ b [+ , [ [* , [ 3 b[[+ , [ t 1]]]]] [u- , [ a]]]]]] , [+ , [ [* , [ [u- , [ 2]] b]] 1]]]] , [ [min,a]]]] , time: 3 step: 1 , [ [A , [ B C]] [A.pay , [ B.day]]]]


Full program
Time horizon : 3
Time step : 1
All the defined nodes : 
	Name : ABCD
		Parameters : [ [c , [

### Matrix  acquisition 

All the previous steps can be called by using main.py.
- The --lex option prints all the Tokens stream 
- The --parse option prints the syntax tree



In [9]:
!python main.py newfile.txt

a[0] b[0] 
a[1] b[1] 
a[2] b[2] 

CONSTRAINT : 0
t : 0
1 0 0 
0 0 0 
t : 1
1 0 0 
0 0 0 
t : 2
1 0 0 
0 0 0 
const : 1
CONSTRAINT : 1
t : 0
-3 1 0 
0 0 0 
t : 1
-3 0 1 
0 0 0 
t : 2
-3 0 0 
0 0 0 
const : 0
CONSTRAINT : 2
t : 0
-3 0 0 
0 1 0 
t : 1
0 -3 0 
0 0 0 
t : 2
0 0 -3 
0 0 0 
const : 0
CONSTRAINT : 3
t : 0
1 0 0 
3 -3 0 
t : 1
0 1 0 
0 3 -3 
t : 2
0 0 1 
0 0 3 
const : 1
CONSTRAINT : 4
t : 0
1 0 0 
3 -3 0 
t : 1
0 1 0 
0 3 -3 
t : 2
0 0 1 
0 0 3 
const : 1
a[0] b[0] 
a[1] b[1] 
a[2] b[2] 

CONSTRAINT : 0
t : 0
1 0 0 
0 0 0 
t : 1
1 0 0 
0 0 0 
t : 2
1 0 0 
0 0 0 
const : 1
CONSTRAINT : 1
t : 0
-3 1 0 
0 0 0 
t : 1
-3 0 1 
0 0 0 
t : 2
-3 0 0 
0 0 0 
const : 0
CONSTRAINT : 2
t : 0
-3 0 0 
0 1 0 
t : 1
0 -3 0 
0 0 0 
t : 2
0 0 -3 
0 0 0 
const : 0
CONSTRAINT : 3
t : 0
1 0 0 
3 -3 0 
t : 1
0 1 0 
0 3 -3 
t : 2
0 0 1 
0 0 3 
const : 1
CONSTRAINT : 4
t : 0
1 0 0 
3 -3 0 
t : 1
0 1 0 
0 3 -3 
t : 2
0 0 1 
0 0 3 
const : 1
